## Summary

In this notebook we load a network trained to reconstruct protein sequences and use this network to design sequences that fit the geometry of CATH domain [1n5uA03](http://www.cathdb.info/version/latest/domain/1n5uA03).

----

## Imports

In [ ]:
import importlib
import logging
import subprocess
from pathlib import Path

import ipywidgets as widgets
import nglview
from IPython.display import HTML, display
from ipywidgets import Layout

import proteinsolver
import proteinsolver.dashboard
import torch
from kmbio import PDB

In [ ]:
%load_ext autoreload
%autoreload 2

## Properties

In [ ]:
NOTEBOOK_NAME = "30_design_dashboard"

NOTEBOOK_PATH = Path(NOTEBOOK_NAME).resolve()
NOTEBOOK_PATH.mkdir(exist_ok=True)

In [ ]:
NOTEBOOK_PATH

In [ ]:
JUPYTER_DATA_PATH = Path(subprocess.check_output(["jupyter", "--data-dir"], universal_newlines=True).strip()).resolve(strict=True)

In [ ]:
JUPYTER_DATA_PATH

In [ ]:
STATIC_DATA_PATH = JUPYTER_DATA_PATH.joinpath("voila", "templates", "mytemplate", "static").resolve(strict=True)

In [ ]:
STATIC_DATA_PATH

In [ ]:
UNIQUE_ID = "191f05de"

In [ ]:
BEST_STATE_FILES = {
    #
    "191f05de": "protein_train/191f05de/e53-s1952148-d93703104.state"
}

In [ ]:
LOGGER = logging.getLogger(__name__)

In [ ]:
global_state = proteinsolver.dashboard.global_state
global_state.output_folder = STATIC_DATA_PATH

## Custom CSS

In [ ]:
HTML("""\
<style>
h1 {
    margin-top: -20px !important;
}

.myheading {
    font-size: large;
}

.mytext > .widget-label {
    font-family: monospace;
    font-size: small;
    width: 60px;
}

.mysequence > .widget-label {
    line-height: 1rem;
    min-height: calc( var(--jp-widgets-inline-height) + 12);
}

.mynglout {
    z-index: -100
}
</style>
""")

## Load model

In [ ]:
Net = importlib.import_module(f"protein_train.{UNIQUE_ID}.model").Net

In [ ]:
global_state.net_class = Net
global_state.state_file = BEST_STATE_FILES[UNIQUE_ID]
global_state.net_kwargs = {"x_input_size": 21, "adj_input_size": 2, "hidden_size": 128, "output_size": 20}

## Design pipeline

### Load protein sequence and geometry

In [ ]:
structure = PDB.load(Path(proteinsolver.__path__[0]).resolve(strict=True).joinpath("data", "inputs", "1n5uA03.pdb"))
proteinsolver.dashboard.load_structure(structure)

### Test ProteinSolverProcess

In [ ]:
data = global_state.data
data.x = torch.tensor(
    [proteinsolver.utils.AMINO_ACID_TO_IDX[aa] for aa in global_state.target_sequence],
    dtype=torch.long,
)

In [ ]:
global_state.proteinsolver_process = proteinsolver.dashboard.ProteinSolverProcess(
    global_state.net_class,
    global_state.state_file,
    global_state.data,
    num_designs=2,
    net_kwargs=global_state.net_kwargs,
)

In [ ]:
global_state.proteinsolver_process.start()

In [ ]:
global_state.generated_sequences = [global_state.proteinsolver_process.output_queue.get() for _ in range(2)]

In [ ]:
global_state.generated_sequences

In [ ]:
global_state.proteinsolver_process.join()

## Dashboard

In [ ]:
ngl_stage = nglview.NGLWidget(height="300px", width="300px")

In [ ]:
ngl_stage_out = widgets.Output(layout=widgets.Layout(min_width="300px", margin="-50px 0px 0px 0px"))
ngl_stage_out.add_class("mynglout")

with ngl_stage_out:
    display(ngl_stage)

In [ ]:
target_selection_out = widgets.Output()

In [ ]:
sequence_generation_out = widgets.Output()

In [ ]:
load_structure_button = proteinsolver.dashboard.create_load_structure_button(
    ngl_stage, target_selection_out, sequence_generation_out
)

In [ ]:
load_example_buttons = proteinsolver.dashboard.create_load_example_buttons(
    ngl_stage, target_selection_out, sequence_generation_out
)

In [ ]:
load_structure_out = widgets.Output(layout=widgets.Layout(width="auto"))

with load_structure_out:
    display(HTML('<p class=myheading>1. Load a reference protein structure</p>'))
    display(HTML('<p>Chose a protein structure to upload:</p>'))
    display(load_structure_button)
    display(HTML('<p style="margin-top: 16px">Alternatively, try one of the examples provided below:</p>'))
    display(load_example_buttons)

# Generate protein sequences matching a predetermined geometry

In [ ]:
out = widgets.VBox(
    [
        widgets.HBox(
            [load_structure_out, ngl_stage_out],
            layout=widgets.Layout(flex_flow="row nowrap", justify_content="space-between"),
        ),
        target_selection_out,
        sequence_generation_out,
    ]
)

In [ ]:
out

In [ ]:
# wg, (
#     num_designs_field,
#     run_ps_button,
#     download_button,
#     gpu_status_out,
#     progress_bar,
#     progress_bar_status,
#     msa_view,
# ) = proteinsolver.dashboard.get_or_create_sequence_generation_widget()

In [ ]:
wg = proteinsolver.dashboard.get_or_create_sequence_generation_widget()

In [ ]:
wg

In [ ]:
# msa_view.clear_output()

In [ ]:
# proteinsolver.dashboard.on_run_ps_button_clicked(run_ps_button, num_designs_field)

In [ ]:
display(HTML("""\
<hr>
<p>Running into issues? Please send an email to <a href="help@proteinsolver.org">help@proteinsolver.org</a>.
<br>
<em>This website works best using the latest versions of Firefox or Chrome web browsers.</em>
</p>
"""))